In [1]:
# Imports
import os
import sys
import numpy as np
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F

import coremltools as ct
from coremltools.models.neural_network import quantization_utils

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [3]:
restore_path = "./model_parameter.pt"
valid_dir = "./kodak/"
y_hat_dir = "./y_hat/"

In [4]:
# Helper Function to get Nearest Weights
def get_nearestup_weight(cin):
    filt=torch.Tensor([[0, 0, 0, 0], [0, 1, 1, 0], [0, 1, 1, 0], [0, 0, 0, 0]])[None, None, ...]
    weight = np.zeros((cin, cin, 4, 4),
                      dtype=np.float64)
    weight[range(cin), range(cin), :, :] = filt
    return torch.from_numpy(weight).float()

In [5]:
class EncoderDecoder(nn.Module):
    def __init__(self):
        super(EncoderDecoder, self).__init__()

        # Define Decoder convolution and activation layers
        self.Conv_D_1 = torch.nn.Conv2d(in_channels=12,  out_channels=192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode='zeros', bias=True)
        self.Conv_D_2 = torch.nn.Conv2d(in_channels=192, out_channels=192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode='zeros', bias=True)
        self.Conv_D_3 = torch.nn.Conv2d(in_channels=192, out_channels=192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode='zeros', bias=True)
        self.Conv_D_4 = torch.nn.Conv2d(in_channels=192, out_channels=3,   kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode='zeros', bias=True)

        self.Act_D_1 = nn.PReLU(init=0.2)
        self.Act_D_2 = nn.PReLU(init=0.2)
        self.Act_D_3 = nn.PReLU(init=0.2)

    def forward(self, y_hat):

        # Decoder: y_hat --> x_hat
        Conv_D_1 = self.Act_D_1(self.Conv_D_1(y_hat))
        #Conv_D_2 = self.Act_D_2(self.Conv_D_2(F.interpolate(Conv_D_1, mode='nearest', scale_factor=2)))
        #Conv_D_3 = self.Act_D_3(self.Conv_D_3(F.interpolate(Conv_D_2, mode='nearest', scale_factor=2)))
        
        # Since, CoreML/ONNX converter Tool does not support Upsample layer,
        # replacing the layer with 2D Transposed Comvolution Layers
        upweight_1 = get_nearestup_weight(Conv_D_1.size(1))
        Conv_D_2 = self.Act_D_2(self.Conv_D_2(F.conv_transpose2d(Conv_D_1, upweight_1.detach(),stride=2, padding=1)))
        
        upweight_2 = get_nearestup_weight(Conv_D_2.size(1))
        Conv_D_3 = self.Act_D_3(self.Conv_D_3(F.conv_transpose2d(Conv_D_2, upweight_2.detach(),stride=2, padding=1)))
        
        x_hat    = self.Conv_D_4(Conv_D_3)

        return x_hat

## Points to Remember

**Ground Truth Image shape** = (1, 3, 512, 768)

**y_hat** represents the test input Image

**y_hat.shape** = (1, 12, 128, 192)

***model takes y_hat as input and returns compressed image.***

**Model Prediction Output Shape** = (1, 3, 512, 768)

Then we calculate the MSE between the Original Image and the Model Predicted Image and print it out.

In [6]:
model = EncoderDecoder().to(device)
model.eval()

# Restore Paths:
print(f'Restoring weights from {restore_path}')
checkpoint = torch.load(restore_path, map_location=device)
model.load_state_dict(checkpoint['weights'])

Restoring weights from ./model_parameter.pt


<All keys matched successfully>

In [7]:
# Create dummy input
dummy_input = torch.rand(1, 12, 128, 192)

# Define input / output names
input_names = ["y_hat"]
output_names = ["pred"]

torch.onnx.export(model,
                  dummy_input,
                  "test01_network_op11.onnx",
                  verbose=True,
                  input_names=input_names,
                  output_names=output_names,
                  opset_version= 11)

<ipython-input-4-bc49a379f27e>:3: TracerWarning: torch.Tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  filt=torch.Tensor([[0, 0, 0, 0], [0, 1, 1, 0], [0, 1, 1, 0], [0, 0, 0, 0]])[None, None, ...]
<ipython-input-4-bc49a379f27e>:4: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  weight = np.zeros((cin, cin, 4, 4),
<ipython-input-4-bc49a379f27e>:6: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. 

graph(%y_hat : Float(1, 12, 128, 192),
      %Conv_D_1.weight : Float(192, 12, 3, 3),
      %Conv_D_1.bias : Float(192),
      %Conv_D_2.weight : Float(192, 192, 3, 3),
      %Conv_D_2.bias : Float(192),
      %Conv_D_3.weight : Float(192, 192, 3, 3),
      %Conv_D_3.bias : Float(192),
      %Conv_D_4.weight : Float(3, 192, 3, 3),
      %Conv_D_4.bias : Float(3),
      %26 : Float(1, 1, 1),
      %27 : Float(1, 1, 1),
      %28 : Float(1, 1, 1)):
  %12 : Float(1, 192, 128, 192) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[3, 3], pads=[1, 1, 1, 1], strides=[1, 1]](%y_hat, %Conv_D_1.weight, %Conv_D_1.bias) # /Users/anujdutt/miniconda3/envs/PyTorch/lib/python3.8/site-packages/torch/nn/modules/conv.py:345:0
  %14 : Float(1, 192, 128, 192) = onnx::PRelu(%12, %26) # /Users/anujdutt/miniconda3/envs/PyTorch/lib/python3.8/site-packages/torch/nn/functional.py:1263:0
  %15 : Float(192, 192, 4, 4) = onnx::Constant[value=<Tensor>]()
  %16 : Float(1, 192, 256, 384) = onnx::ConvTranspose[dil

In [8]:
# Convert from ONNX to Core ML
coreml_model  = ct.converters.onnx.convert(model='test01_network_op11.onnx', minimum_ios_deployment_target='13')

1/9: Converting Node Type Conv
2/9: Converting Node Type PRelu
3/9: Converting Node Type ConvTranspose
4/9: Converting Node Type Conv
5/9: Converting Node Type PRelu
6/9: Converting Node Type ConvTranspose
7/9: Converting Node Type Conv
8/9: Converting Node Type PRelu
9/9: Converting Node Type Conv
Translation to CoreML spec completed. Now compiling the CoreML model.
Model Compilation done.


In [9]:
coreml_model.input_description['y_hat'] = 'Input Image'
coreml_model.output_description['pred'] = 'Compressed Image Output'
coreml_model.short_description = 'FP-32 model.'

In [10]:
coreml_model

input {
  name: "y_hat"
  shortDescription: "Input Image"
  type {
    multiArrayType {
      shape: 1
      shape: 12
      shape: 128
      shape: 192
      dataType: FLOAT32
    }
  }
}
output {
  name: "pred"
  shortDescription: "Compressed Image Output"
  type {
    multiArrayType {
      shape: 1
      shape: 3
      shape: 512
      shape: 768
      dataType: FLOAT32
    }
  }
}
metadata {
  shortDescription: "FP-32 model."
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "onnx==1.7.0"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "4.0b1"
  }
}

In [11]:
# Save the Model FP 32
coreml_model.save('./test01_model.mlmodel')

## Model Quantization

Quantize to: 16 and 8 bits

In [12]:
# Function to Quantize and Save the Models
def quantize_model(base_mode=None, quantization_bits=None):
    quantized_model = quantization_utils.quantize_weights(base_mode, nbits=quantization_bits)
    save_path = './test01_model_' + str(quantization_bits) + 'bit.mlmodel'
    quantized_model.short_description = str(quantization_bits) + ' bit quantized model.'
    quantized_model.save(save_path)

In [13]:
# Quantize the Model
quant_bits = [16, 8, 4, 2, 1]

for bit in quant_bits:
    quantize_model(coreml_model, bit)

Quantizing using linear quantization
Quantizing layer Conv_0
Quantizing layer ConvTranspose_3
Quantizing layer Conv_4
Quantizing layer ConvTranspose_7
Quantizing layer Conv_8
Quantizing layer Conv_10
Quantizing using linear quantization
Optimizing Neural Network before Quantization:
Finished optimizing network. Quantizing neural network..
Quantizing layer Conv_0
Quantizing layer ConvTranspose_3
Quantizing layer Conv_4
Quantizing layer ConvTranspose_7
Quantizing layer Conv_8
Quantizing layer Conv_10
Quantizing using linear quantization
Optimizing Neural Network before Quantization:
Finished optimizing network. Quantizing neural network..
Quantizing layer Conv_0
Quantizing layer ConvTranspose_3
Quantizing layer Conv_4
Quantizing layer ConvTranspose_7
Quantizing layer Conv_8
Quantizing layer Conv_10
Quantizing using linear quantization
Optimizing Neural Network before Quantization:
Finished optimizing network. Quantizing neural network..
Quantizing layer Conv_0
Quantizing layer ConvTransp

## CoreML Model Testing

## CoreML Model Testing

1. Load the CoreML Model
2. Loop through the data and test for MSE

**GT image shape:**  (1, 3, 512, 768)

**y_hat shape:**  torch.Size([1, 12, 512, 768])

**Model prediction shape:**  (1, 3, 512, 768)

In [14]:
def test_coreml_model(quantization=32):
    if quantization == 32:
        ml_model = ct.models.MLModel('./test01_model.mlmodel')
    elif quantization == 16:
        ml_model = ct.models.MLModel('./test01_model_16bit.mlmodel')
    elif quantization == 8:
        ml_model = ct.models.MLModel('./test01_model_8bit.mlmodel')
    elif quantization == 4:
        ml_model = ct.models.MLModel('./test01_model_4bit.mlmodel')
    elif quantization == 2:
        ml_model = ct.models.MLModel('./test01_model_2bit.mlmodel')
    elif quantization == 1:
        ml_model = ct.models.MLModel('./test01_model_1bit.mlmodel')
    
    print(ml_model)

    tmp = []
    
    filelist_valid = np.sort([file for file in os.listdir(valid_dir) if file.endswith('.png')])

    for j in range(0, len(filelist_valid)):
        image = cv2.imread(valid_dir + filelist_valid[j]).astype(np.float32) / 255.0
        image = np.expand_dims(np.transpose(image, [2,0,1]), axis=0)
        print("GT image shape: ", image.shape)

        y_hat = np.load(y_hat_dir + filelist_valid[j][:-4] + ".npy")
        print("y_hat shape: ", y_hat.shape)

        pred = ml_model.predict({'y_hat': y_hat})

        model_prediction = np.asarray(pred['pred'])
        print("Model prediction shape: ", model_prediction.shape)

        mse = np.mean((image - model_prediction) ** 2) * 255.0 ** 2
        print(f"Image: {filelist_valid[j]}, MSE: {mse}")
        tmp.append(mse)
        
        print("\n\n")
    
    print("Model Quantization: ", quantization)
    print("MSE Values: ",tmp)

In [15]:
# Test all Models
test_coreml_model(quantization = 32)

input {
  name: "y_hat"
  shortDescription: "Input Image"
  type {
    multiArrayType {
      shape: 1
      shape: 12
      shape: 128
      shape: 192
      dataType: FLOAT32
    }
  }
}
output {
  name: "pred"
  shortDescription: "Compressed Image Output"
  type {
    multiArrayType {
      shape: 1
      shape: 3
      shape: 512
      shape: 768
      dataType: FLOAT32
    }
  }
}
metadata {
  shortDescription: "2 bit quantized model."
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "onnx==1.7.0"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "4.0b1"
  }
}

GT image shape:  (1, 3, 512, 768)
y_hat shape:  (1, 12, 128, 192)
Model prediction shape:  (1, 3, 512, 768)
Image: 0.png, MSE: 67499445093.75



GT image shape:  (1, 3, 512, 768)
y_hat shape:  (1, 12, 128, 192)
Model prediction shape:  (1, 3, 512, 768)
Image: 1.png, MSE: 59796343814.0625



GT image shape:  (1, 3, 512, 768)
y_hat shape:  (1, 12, 128, 192)
Model predictio